In [9]:
import datetime
import logging
import os
import glob 
from datetime import datetime, timedelta

from pathlib import Path
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from pyproj import Transformer
from sklearn.metrics import cohen_kappa_score
from shapely.geometry import LinearRing, Point, Polygon, box
import rasterio
import requests
from pprint import pprint

In [13]:
def download_dswx_hls(polys, 
                      datetime_start_str, 
                      datetime_end_str, 
                      MGRS_tile,
                      download_folder,
                      download_flag=True):
    CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
    url = f'{CMR_OPS}/{"granules"}'
    boundind_box = polys.bounds
    provider = 'POCLOUD'
    parameters = {'temporal': f'{datetime_start_str},{datetime_end_str}',
                    'concept_id': 'C2617126679-POCLOUD', #HLS
                    #'concept_id': 'C2949811996-POCLOUD', #S1
                    'provider': provider,
                    #'bounding_box': f'{boundind_box[1]+0.2},{boundind_box[0]+0.2},{boundind_box[3]-0.2},{boundind_box[2]-0.2}',
                    'bounding_box': f'{boundind_box[1]},{boundind_box[0]},{boundind_box[3]},{boundind_box[2]}',
                'page_size': 200,}
    print(parameters)
    response = requests.get(url,
                            params=parameters,
                            headers={
                                'Accept': 'application/json'
                            }
                        )
    print(response.status_code)
    print(response.headers['CMR-Hits'])
    downloaded_list = []
    num_search_data = response.headers['CMR-Hits']
    number_hls_data = 0
    if num_search_data:
        collections = response.json()['feed']['entry']
        # print('collection', collections)
        for collection in collections:
            dswx_hls_file_id = collection['producer_granule_id']
            if MGRS_tile in dswx_hls_file_id:
                print('url', f'{collection["links"][4]["href"]}')
                print('s3', f'{collection["links"][3]["href"]}')
                if download_flag:
                    for layer in [2, 4, 6]:  # 2: WTR, 4: BWTR, 6:CONF
                        dswx_hls_url = collection["links"][layer]["href"]
                        dswx_hls_filename = os.path.basename(dswx_hls_url)
                        download_file = f'{download_folder}/{dswx_hls_filename}'

                        if not os.path.isfile(download_file):
                            response = requests.get(dswx_hls_url, stream=True)

                        # Check if the request was successful
                            if response.status_code == 200:
                                # Open a local file with wb (write binary) permission.
                                with open(f'{download_file}', 'wb') as file:
                                    print('downloading')
                                    for chunk in response.iter_content(chunk_size=128):
                                        file.write(chunk)
                else:
                    print('under dev.')
                downloaded_list.append(download_file)
                number_hls_data += 1

            else:
                print('MGRS tile id does not match')

    return number_hls_data, downloaded_list

In [14]:
#poly_cand = box(64.90, -148,  65,-147)
poly_cand = box(37.9,-123.1,  38.0,-121.9)

datetime_start_str = '2024-10-01T00:00:00Z'
datetime_end_str = '2024-10-13T00:00:00Z'
#mgrs_tile = '06WWT'
mgrs_tile = '10SEH'
download_folder = '/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_' + mgrs_tile
# formats_output = '%Y-%m-%dT%H:%M:%SZ'

download_dswx_hls(poly_cand, datetime_start_str, datetime_end_str, mgrs_tile, download_folder)  

{'temporal': '2024-10-01T00:00:00Z,2024-10-13T00:00:00Z', 'concept_id': 'C2949811996-POCLOUD', 'provider': 'POCLOUD', 'bounding_box': '-123.1,37.9,-121.9,38.0', 'page_size': 200}
200
24
MGRS tile id does not match
MGRS tile id does not match
MGRS tile id does not match
MGRS tile id does not match
MGRS tile id does not match
url https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T10SEH_20241003T020832Z_20241003T103817Z_S1A_30_v1.0_B02_BWTR.tif
s3 s3://podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T10SEH_20241003T020832Z_20241003T103817Z_S1A_30_v1.0_B01_WTR.tif
downloading
downloading
downloading
MGRS tile id does not match
MGRS tile id does not match
url https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T10SEH_20241003T140755Z_20241003T211205Z_S1A_30_v1.0_B02_BWTR.tif
s3 s3://podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T10SEH_202410

(4,
 ['/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_10SEH/OPERA_L3_DSWx-S1_T10SEH_20241003T020832Z_20241003T103817Z_S1A_30_v1.0_B03_CONF.tif',
  '/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_10SEH/OPERA_L3_DSWx-S1_T10SEH_20241003T140755Z_20241003T211205Z_S1A_30_v1.0_B03_CONF.tif',
  '/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_10SEH/OPERA_L3_DSWx-S1_T10SEH_20241008T141619Z_20241008T201644Z_S1A_30_v1.0_B03_CONF.tif',
  '/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_10SEH/OPERA_L3_DSWx-S1_T10SEH_20241010T020037Z_20241010T184256Z_S1A_30_v1.0_B03_CONF.tif'])

In [36]:
def download_dswx_s1(polys, 
                      datetime_start_str, 
                      datetime_end_str, 
                      MGRS_tile,
                      download_folder,
                      download_flag=True):
    CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
    url = f'{CMR_OPS}/{"granules"}'
    boundind_box = polys.bounds
    provider = 'POCLOUD'
    parameters = {'temporal': f'{datetime_start_str},{datetime_end_str}',
                    #'concept_id': 'C2617126679-POCLOUD', #HLS
                    'concept_id': 'C2949811996-POCLOUD', #S1
                    'provider': provider,
                    #'bounding_box': f'{boundind_box[1]+0.2},{boundind_box[0]+0.2},{boundind_box[3]-0.2},{boundind_box[2]-0.2}',
                    'bounding_box': f'{boundind_box[1]},{boundind_box[0]},{boundind_box[3]},{boundind_box[2]}',
                'page_size': 200,}
    print(parameters)
    response = requests.get(url,
                            params=parameters,
                            headers={
                                'Accept': 'application/json'
                            }
                        )
    print(response.status_code)
    print(response.headers['CMR-Hits'])
    downloaded_list = []
    num_search_data = response.headers['CMR-Hits']
    number_hls_data = 0
    if num_search_data:
        collections = response.json()['feed']['entry']
        # print('collection', collections)
        for collection in collections:
            dswx_hls_file_id = collection['producer_granule_id']
            if MGRS_tile in dswx_hls_file_id:
                print('url', f'{collection["links"][4]["href"]}')
                print('s3', f'{collection["links"][3]["href"]}')
                if download_flag:
                    for layer in [2, 4, 6]:  # 2: WTR, 4: BWTR, 6:CONF
                        dswx_hls_url = collection["links"][layer]["href"]
                        dswx_hls_filename = os.path.basename(dswx_hls_url)
                        download_file = f'{download_folder}/{dswx_hls_filename}'

                        if not os.path.isfile(download_file):
                            response = requests.get(dswx_hls_url, stream=True)

                        # Check if the request was successful
                            if response.status_code == 200:
                                # Open a local file with wb (write binary) permission.
                                with open(f'{download_file}', 'wb') as file:
                                    print('downloading')
                                    for chunk in response.iter_content(chunk_size=128):
                                        file.write(chunk)
                else:
                    print('under dev.')
                downloaded_list.append(download_file)
                number_hls_data += 1

            else:
                print('MGRS tile id does not match')

    return number_hls_data, downloaded_list

In [37]:
#poly_cand = box(64.90, -148,  65,-147)
poly_cand = box(37.9,-123.1,  38.0,-121.9)

datetime_start_str = '2024-10-01T00:00:00Z'
datetime_end_str = '2024-10-13T00:00:00Z'
#mgrs_tile = '06WWT'
mgrs_tile = '10SEH'
download_folder = '/u/aurora-r0/jeon/tools/DSWx-fusion/DSWx-Enhanced-Analysis/download_' + mgrs_tile
# formats_output = '%Y-%m-%dT%H:%M:%SZ'

download_dswx_s1(poly_cand, datetime_start_str, datetime_end_str, mgrs_tile, download_folder)  

{'temporal': '2024-10-01T00:00:00Z,2024-10-13T00:00:00Z', 'concept_id': 'C2949811996-POCLOUD', 'provider': 'POCLOUD', 'bounding_box': '-148.0,64.9,-147.0,65.0', 'page_size': 200}
200
11
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T06WWS_20241007T032104Z_20241007T102813Z_S1A_30_v1.0_B02_BWTR.tif
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T06WVS_20241007T032104Z_20241007T102813Z_S1A_30_v1.0_B02_BWTR.tif
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T06WWT_20241007T032203Z_20241007T113244Z_S1A_30_v1.0_B02_BWTR.tif
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T06WVT_20241007T032203Z_20241007T113244Z_S1A_30_v1.0_B02_BWTR.tif
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T06WVT_202

'11'